# CleaningModel
### TC2008B. Modelación de sistemas multiagentes con gráficas computacionales
José Emiliano Riosmena Castañón - A01704245

In [15]:
# Mesa imports
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Statistic libraries imports
import numpy as np
import pandas as pd

# Other imports
import time
import datetime
import random

In [16]:
class CleaningAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
    
    def move(self):
        # Get neighboring positions (Moore neighborhood) excluding the current position
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Filter out positions that are not empty
        possible_steps = [i for i in possible_steps if self.model.grid.is_cell_empty(i)]

        # Check if there are any possible steps
        if possible_steps:
            # Choose a random position from the list of possible steps
            new_position = random.choice(possible_steps)
            # Move the agent to the new position
            self.model.grid.move_agent(self, new_position)

    def step(self):
        # Get current position
        x, y = self.pos

        # Check if the current position is clean
        if self.model.is_clean(x, y):
            # If clean, move to a new position
            self.move()
        else:
            # If not clean, clean the current position and then move to a new position
            self.model.clean(x, y)
            self.move()


In [17]:
class CleaningModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells):
        # Initialize the grid and scheduler
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        
        # Initialize counters for dirty and clean cells, steps, and set the number of dirty cells
        self.dirty = np.zeros((width, height))
        self.dirty_cells = 1
        self.clean_cells = 0
        self.steps = 0

        # Initialize agent IDs starting from 0
        id = 0

        # Create CleaningAgent instances, add them to the schedule, and place them on the grid
        for i in range(num_agents):
            agent = CleaningAgent(id, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (1, 1))
            id += 1

        # Calculate the total number of cells and set the number of dirty cells
        total_cells = width * height
        self.dirty_cells = int(total_cells * dirty_cells)

        # Randomly set dirty cells on the grid
        for i in range(self.dirty_cells):
            x, y = self.random_empty_cell()
            self.dirty[x][y] = 1

    def step(self):
        # Advance the model by one step, updating agents and model state
        self.schedule.step()
        self.steps += 1

    def clean(self, x, y):
        # Mark a cell as clean and update clean cell count
        self.dirty[x][y] = 0
        self.clean_cells += 1

    def is_clean(self, x, y):
        # Check if a cell is clean
        return self.dirty[x][y] == 0

    def random_empty_cell(self):
        # Get a random empty cell
        x, y = self.random_pos()

        # Keep selecting a random position until an empty cell is found
        while not self.grid.is_cell_empty((x, y)) or self.dirty[x][y] == 1:
            x, y = self.random_pos()

        return x, y

    def random_pos(self):
        # Generate a random position within the grid boundaries
        return random.randrange(self.grid.width), random.randrange(self.grid.height)

    def get_steps(self):
        # Get the current number of steps taken
        return self.steps

    def get_dirty_cells(self):
        # Get the total number of dirty cells
        return self.dirty_cells

    def get_clean_cells(self):
        # Get the total number of clean cells
        return self.clean_cells

    def get_dirty_percentage(self):
        # Calculate and return the percentage of dirty cells in the grid
        return round(100 - (self.clean_cells / self.dirty_cells) * 100, 2)

In [40]:
# Set the dimensions of the environment
WIDTH = 100
HEIGHT = 100
# Set the number of cleaning agents
NUM_AGENTS = 2
# Set the percentage of dirty cells in the environment
DIRT_PERCENTAGE = 0.9
# Set the maximum number of steps for the cleaning process
MAX_STEPS = 10000

# Record the start time to measure the execution time of the cleaning process
start_time = time.time()

# Call the clean function with the specified parameters
model = CleaningModel(WIDTH, HEIGHT, NUM_AGENTS, DIRT_PERCENTAGE)

# Print the initial percentage of dirty cells
print("Initial dirty percentage: ", model.get_dirty_percentage())

# Run the cleaning process until the environment is completely clean or the maximum number of steps is reached
for i in range(MAX_STEPS):
    if model.get_dirty_percentage() == 0:
        break

    model.step()

# Calculate the elapsed time for the cleaning process
end_time = time.time() - start_time

# Print the results, including the time taken and the final result of the cleaning process
print("Final dirty percentage: ", model.get_dirty_percentage())
print("Time taken: ", end_time)
print("Steps taken: ", model.get_steps())


Initial dirty percentage:  100.0
Final dirty percentage:  53.51
Time taken:  0.21248459815979004
Steps taken:  10000


## Preguntas
#### ¿Cuántos pasos de simulación toma limpiar todo el espacio?
Aproximadamente 274080 pasos con un agente. Y 154180 con dos agentes.
#### ¿Qué porcentaje de celdas sucias queda con los siguientes pasos de simulación: 100, 1000, 10000?
100: 99.44% (1 agente), 98.82% (2 agentes)

1000: 96.02% (1 agente), 92.34% (2 agentes)

10000: 70.6% (1 agente), 53.51% (2 agentes)